# Elastic search config

### Test connection

In [ ]:
import requests
from requests.auth import HTTPBasicAuth

# Elasticsearch connection settings
es_url = "http://localhost:9201"
es_user = "elastic"
es_password = "apppw"

try:
    response = requests.get(f"{es_url}/", auth=HTTPBasicAuth(es_user, es_password), timeout=5)
    if response.status_code == 200:
        print("Successfully connected to Elasticsearch!")
    else:
        print(f"Failed to connect to Elasticsearch: {response.status_code} {response.text}")
except Exception as e:
    print(f"Error connecting to Elasticsearch: {e}")

### Create empty Index

In [ ]:
from elasticsearch import Elasticsearch

# Connection
es_url = "http://localhost:9201"
es_user = "elastic"
es_password = "apppw"
index_name = "my_index"

es = Elasticsearch(
    es_url,
    basic_auth=(es_user, es_password)
)

# 1. Delete index if it already exists
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    print(f"Deleted old index: {index_name}")

# 2. Define settings + mappings
settings = {
    "settings": {
        "analysis": {
            "analyzer": {
                "ngram_analyzer": {
                    "tokenizer": "ngram_tokenizer"
                }
            },
            "tokenizer": {
                "ngram_tokenizer": {
                    "type": "ngram",
                    "min_gram": 3,
                    "max_gram": 4,
                    "token_chars": ["letter", "digit"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "title": {
                "type": "text",
                "fields": {
                    "standard": {"type": "text", "analyzer": "standard"},
                    "ngram": {"type": "text", "analyzer": "ngram_analyzer"}
                }
            },
            "year": {"type": "integer"},
            "genre": {"type": "keyword"}
        }
    }
}


# 3. Create index
es.indices.create(index=index_name, body=settings)
print(f"Created index: {index_name}")


### Fill index

In [ ]:
from elasticsearch import Elasticsearch, helpers
import pandas as pd
import numpy as np
import time

# Elasticsearch connection
es_url = "http://localhost:9201"
es_user = "elastic"
es_password = "apppw"
index_name = "my_index"

es = Elasticsearch(es_url, basic_auth=(es_user, es_password))

# Load CSV
csv_file = "products-2000000.csv"  # or larger CSV
df = pd.read_csv(csv_file)

# Replace NaN with None
df = df.where(pd.notnull(df), None)

# Rename columns if necessary (match ES mapping)
# For example: 'Title', 'Year', 'Genre'
# Make sure the column names match the mappings
# df.rename(columns={"OldName": "NewName"}, inplace=True)

# Convert DataFrame to dicts for ES
records = df.to_dict(orient="records")

batch_size = 1000  # tweak based on memory / ES capacity
start_time = time.time()

print("Starting to insert rows")

for start in range(0, len(records), batch_size):
    end = start + batch_size
    batch = records[start:end]
    actions = [
        {
            "_index": index_name,
            "_source": rec
        }
        for rec in batch
    ]
    helpers.bulk(es, actions)

elapsed = time.time() - start_time
print(f"Finished inserting {len(records)} rows into '{index_name}' in {elapsed:.2f} seconds")
